# Redditing

This notebook uses the `praw` library to download the 100 hot posts in `r/conspiracy` and save them to a dataframe (and later to a CSV file). It then uses the ID for each post to grab all the comments for a post. It adds those to the dataframe, which can then be saved as a CSV.

In [2]:
import os
import praw
import pandas as pd
import numpy as np

# SECRET DECODER RING
webber_id = os.getenv("REDDIT_API_ID")
webber_secret = os.getenv("REDDIT_API_SECRET")

In [3]:
reddit = praw.Reddit(
    client_id="",
    client_secret="",
    user_agent="webber"
    )
print(reddit.read_only)
# try:
#     print("Authenticated as {}".format(reddit.user.me()))
# except ResponseException:
#     print("Something went wrong during authentication")

# print(reddit.auth.url(scopes=["identity"], state="...", duration="permanent"))

True


In [3]:
# hot_posts = reddit.subreddit('conspiracy').hot(limit=5)
# for post in hot_posts:
#     print(post.title)

In [29]:
conspiracy = reddit.subreddit('conspiracy')

posts = []
for post in conspiracy.hot(limit=100):
    posts.append([post.title, 
                  post.score, 
                  post.id, 
                  post.url, 
                  post.num_comments, 
                  post.selftext, 
                  post.created])

posts = pd.DataFrame(posts,
                     columns=['title', 'score', 'id', 'url', 'num_comments', 'body', 'created'])

posts.shape

(100, 7)

In [30]:
posts.head()

,title,score,id,url,num_comments,body,created
0,Our justice system is broken?,1793,13rc2x0,https://i.redd.it/2i041y8gdz1b1.jpg,672,,1.685004e+09
1,China announces new Covid variant. Y’all ready...,417,13rjky7,https://i.redd.it/jdiygs9c612b1.jpg,241,,1.685026e+09
2,Sudden Infant Death Syndrome... because infant...,542,13rf3pz,https://i.redd.it/u9xw4o3p602b1.jpg,340,,1.685014e+09
3,"Under Elon Musk, Twitter has approved 83% of c...",65,13rn9tu,https://english.elpais.com/international/2023-...,74,,1.685034e+09
4,US Default=Chaos & Violence Unleashed! Corpora...,385,13r87u9,https://i.redd.it/pyif95v0qw1b1.png,221,,1.684990e+09


In [45]:
# posts.to_csv('c2.csv')

In [31]:
print(posts[10:11])

                                                title  score       id   
10  Experts Baffled. British Heart Foundation : 50...     34  13riun8  \

                                                  url  num_comments body   
10  https://rumble.com/v2pthn4-astonishing-rise-in...            17       \

         created  
10  1.685024e+09  


In [32]:
posts.at[10,'body']

''

In [33]:
bodies = posts.body.tolist()
text = ' '.join(bodies)
len(text)

36343

In [35]:
post_ids = posts.id.tolist()

print(post_ids[0:10])

['13rc2x0', '13rjky7', '13rf3pz', '13rn9tu', '13r87u9', '13qzit5', '13rd0n9', '13rlv5c', '13r5od6', '13qxjw2']


In [36]:
# Revision to get all comments out due to change in praw library
# https://stackoverflow.com/questions/39840686/attributeerror-class-praw-objects-morecomments-has-no-attribute-body

def get_comments(post_id):
    comments = []
    submission = reddit.submission(id=post_id)
    submission.comments.replace_more(limit=0) # flatten tree
    for comment in submission.comments.list():
        comments.append(comment.body)
    return comments

In [37]:
# with testing done, let's do this
comments = [get_comments(item) for item in post_ids]

In [41]:
# Originally everything went straight into the dataframe
# posts['comments'] = np.array(comments, dtype=object)
# And I used the code below to clean the Meta comment
# match = '[\'###[Meta] Sticky Comment\\n\\n[Rule 2](https://www.reddit.com/r/conspiracy/wiki/faq#wiki_2_-_address_the_argument.3B_not_the_user.2C_the_mods.2C_or_the_sub.) ***does not apply*** when replying to this stickied comment.  \\n\\n[Rule 2](https://www.reddit.com/r/conspiracy/wiki/faq#wiki_2_-_address_the_argument.3B_not_the_user.2C_the_mods.2C_or_the_sub.) ***does apply*** throughout the rest of this thread.\\n\\n*What this means*: Please keep any "meta" discussion directed at specific users, mods, or /r/conspiracy in general in this comment chain ***only.***\\n\\n*I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/conspiracy) if you have any questions or concerns.*\','
# clean = [item.replace(match, '') for item in comments]

for item in comments:
    item.pop(0)

In [42]:
df = pd.DataFrame({'name': post_ids, 'lists': comments})

In [43]:
df.head()

,name,lists
0,13rc2x0,"[Always has been.., Wait until you learn how c..."
1,13rjky7,"[😂 I dont believe shit until it happens, and e..."
2,13rf3pz,[The Japanese started a SIDS awareness campaig...
3,13rn9tu,[A true republican.\n\nCan't get enough of tha...
4,13r87u9,"[I think I get taxed around 35%, I sell pest c..."


In [44]:
# df.to_csv('c2comments.csv')

In [ ]:
# Alternate way to save comment list:
# with open('comments.txt', mode='w') as myfile:
#     myfile.write('\n'.join(comments))